In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/goodreadsbooks/books.csv', error_bad_lines = False)
# 에러가 발생하는 행들은 제외되었습니다.

# **Goodreads API 데이터를 이용한 도서 리스트 분석**
## - Seaborn을 활용한 데이터 시각화
> Goodreads는 아마존(Amazon) 소유의 도서 정보 및 이용자 평가를 무료로 볼 수 있는 사이트이다. (https://www.goodreads.com/)

**Columns Description:**
* bookID
* title
* authors
* average_rating
* isbn
* isbn13
* language_code
* num_pages
* ratings_count
* text_reviews_count
* publication_date
* publisher

In [ ]:
df.head()

In [ ]:
print(df.shape)

데이터셋은 11,123개의 행(Row)과 12개의 열(Column)으로 이루어져 있습니다.

In [ ]:
df.isnull().sum()

결측값은 없네요.

describe를 뽑아 보겠습니다.

In [ ]:
df.describe(include='all')

unique값을 확인해보면, ISBN의 count값과 unique값이 같은 것을 보아 중복되는 도서는 없는 것을 알 수 있습니다.\
하지만 title은 값이 다른 것을 보면 다른 책이지만 제목이 같은 책들이 조금 있는 것 같습니다.\
저자와 출판사는 중복되는 값들이 꽤 많은 것 같습니다.
language_code는 27개가 있네요.


## 1. 평점 상위 Top10 도서

In [ ]:
print('평점의 평균값은 %.2f, 최고, 최저값은 각각 %d, %d입니다.' %(df.average_rating.mean(),df.average_rating.max(),df.average_rating.min()))

In [ ]:
plt.figure(1, figsize=(16,7))
sns.distplot(df.average_rating, color='y')
plt.show()

평점은 4점대가 가장 많은 분포를 가지고 있는 것을 확인할 수 있습니다.\
최고점은 5점이지만, 별점 하나만으로 판단할 수는 없으니, 별점을 준 개수가 많은 것 중에서 별점이 높은 것을 '평점 상위 Top10'으로 선정해보겠습니다.

### 1-1. 평점 개수가 많은 도서 Top10

In [ ]:
plt.style.use('ggplot')
rate_count_top10 = df.sort_values('ratings_count',ascending=False).head(20)
sns.barplot(rate_count_top10['ratings_count'],rate_count_top10['title'].head(10), palette='pastel')
plt.show()

### 1-2. 텍스트 리뷰 개수가 많은 도서 Top10

In [ ]:
textreview_top10 = df.sort_values('text_reviews_count',ascending=False).head(10)
sns.barplot(textreview_top10['text_reviews_count'],textreview_top10['title'], palette='pastel')
plt.show()

### 1-3. 평점개수 평균이상에서 평점 상위 Top10 도서

In [ ]:
print('평점의 개수의 평균은 %d개 이고, 가장 많은 평점 개수는 %d개 입니다.' %(df.ratings_count.mean(),df.ratings_count.max()))

평점 개수 평균인 17942개 이상의 평점을 받은 도서 중 높은 평점을 받은 도서를 알아보겠습니다.

In [ ]:
rate_top10 = df[df.ratings_count > 17942][:10].sort_values(by=['average_rating'], ascending=False)
sns.barplot(rate_top10['average_rating'],rate_top10['title'], palette='pastel')
plt.show()

평점 개수가 많은 것(평균 이상)들 중 평점이 가장 좋은 도서들은 해리포터 시리즈가 상위권을 차지하고 있는 것을 볼 수 있습니다.

### 1-4. 평점이 높은 책들의 평균 페이지 수

Describe에서 페이지 수의 평균은 336p, 최대 페이지 수는 6576p, 최소 페이지 수는 0p인 것으로 확인됩니다.\
(0page의 도서는 페이지를 알 수 없는 도서일까요..) 페이지 수가 0인 도서들을 먼저 확인해보겠습니다.

In [ ]:
# date칼럼에서 연도 가져오기
date = df['publication_date'].str.split('/')
df['year'] = None; df['month'] = None
for i in range(len(df)):
    df['year'][i] = date[i][2]
    df['month'][i] = date[i][0]

In [ ]:
page_zero = df[df['  num_pages']==0]
print('페이지가 0인 도서는 총 %d권 입니다.' %(page_zero['bookID'].count()))
page_zero.head(5)

In [ ]:
plt.figure(1, figsize=(15,7))
sns.countplot(x='year', data=page_zero, palette='pastel')
plt.show()

옛날 책들이라 페이지 수가 기재되지 않았나 생각했는데, 출판연도를 그래프로 확인해보니 오래된 책들만 있지는 않은 것 같습니다.\
무슨 이유인지 2006년도 도서가 많네요. 2006년도에 출판된 다비치코드와 같은 유명한 책도 있는 것을 보니 그냥 페이지 수가 입력되지 않은 책들인 것 같습니다.

평점 상위 도서들의 평균 페이지 수를 알아보겠습니다.

In [ ]:
print('평점 상위 Top10의 평균 페이지 수는 %dp입니다.' %(rate_top10['  num_pages'].mean()))

전체 도서의 평균 페이지 수인 336p보다 훨씬 평균 페이지 수가 높습니다. 아마도 페이지 수가 많은 해리포터 책들이 있어 그런 것 같습니다.

In [ ]:
print('평점 개수가 평균 이상인 도서들의 수는 %d권 이고, 이들의 평균 페이지 수는 %dp이고, 가장 적은 페이지 수는 %dp입니다.'
      %(df[df.ratings_count > 17942]['bookID'].count(),df[df.ratings_count > 17942].sort_values(by=['average_rating'], ascending=False)['  num_pages'].mean(),
       df[df.ratings_count > 17942].sort_values(by=['average_rating'], ascending=False)['  num_pages'].min()))

상위 10개가 아닌 평점 개수 평균 이상의 도서들에서 페이지 수 평균은 위와 같이 전체 평균보다 조금 더 높은 페이지 수를 보이네요.

## 2. 평점 상위 Top10 저자

### 2-1. 평점 개수가 많은 저자 Top10

평점 개수가 많은 저자 별로 Top10을 뽑으면 아래와 같지만, 이름이 다르게 기재되어 있는 경우가 있기 때문에 정확한 파악은 아닐 것 같습니다.

In [ ]:
sns.barplot(rate_count_top10['ratings_count'],rate_count_top10['authors'].head(15), palette='pastel')
plt.show()

1위에 있는 스테프니 메이어(Stephenie Meyer)는 트와일라잇의 저자입니다.

### 2-2. 텍스트 리뷰 개수가 많은 저자 Top10

In [ ]:
sns.barplot(textreview_top10['text_reviews_count'],textreview_top10['authors'], palette='pastel')
plt.show()

### 2-3. 평점개수 평균이상에서 평점 상위 Top10 저자

In [ ]:
rate_top20 = df[df.ratings_count > 17942][:20].sort_values(by=['average_rating'], ascending=False)
sns.barplot(rate_top20['average_rating'],rate_top20['authors'], palette='pastel')
plt.show()

평점 개수로는 트와일라잇의 저자가 1위를 차지했지만, 평점 상위권에서는 해리포터의 저자 조앤롤링이 차지하는 것을 볼 수 있습니다.

### 2-4. 책을 가장 많이 쓴 저자 Top10

In [ ]:
plt.figure(1, figsize=(13,5))
ax = sns.countplot(x='authors', order=df.authors.value_counts().index[0:10], data=df, palette='pastel')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha='right')
plt.show()

## 3. 언어 코드

Describe에서 언어 코드가 27개로 구성되어 있는 것을 볼 수 있었습니다. 어떤 언어들이 차지하고 있는지 확인해보겠습니다.

In [ ]:
df.language_code.value_counts()

In [ ]:
plt.figure(1, figsize=(16,7))
ax = sns.countplot(df['language_code'], palette='pastel')

for i in ax.patches:
    ax.annotate("%.0f" %i.get_height(), (i.get_x()+i.get_width()/2, i.get_height()+130), ha='center', va='center', color='gray')

영어가 압도적으로 많은 것을 확인할 수 있습니다. 다음으로는 스페인어와 프랑스어 순인 것 같습니다.

## 4. 출판연도

In [ ]:
plt.figure(1, figsize=(16,7))
sns.distplot(df.year)
plt.show()

2000년대 출판도서가 가장 많은 비중을 차지하고 있습니다.